## Part 1

In [61]:
struct Maze
    m::Array{Char, 2}
end

Maze(s::Vector{T}) where T <: String = Maze(hcat(collect.(s)...))

find_element(m::Maze, el::Char) = get(CartesianIndices(m.m)[m.m .== el], 1, nothing)
find_elements(m::Maze, els) = [(x, find_element(m, x)) for x in els if find_element(m, x) != nothing]
mkeys(m::Maze) = Dict(find_elements(m, 'a':'z'))
doors(m::Maze) = Dict(find_elements(m, 'A':'Z'))
hero(m::Maze) = find_element(m, '@')
heros(m::Maze) = CartesianIndices(m.m)[m.m .== '@']

function neighbours(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[m.m[l + d] != '#' for d in dirs]]]
end

function floodfill(m::Maze, start)
    arr = fill(-1, size(m.m)...)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function get_door_keys(maze::Maze, arr, start, finish)
    res1 = Char[]
    res2 = Char[]
    current = start
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    while current != finish
        for d in dirs
            if arr[current + d] + 1 == arr[current]
                current = current + d
                if maze.m[current] in collect('A':'Z') push!(res1, lowercase(maze.m[current])) end
                if maze.m[current] in collect('a':'z') push!(res2, lowercase(maze.m[current])) end
                break
            end
        end
    end
                
    res1, res2
end

function build_info(m::Maze)
    res = Dict()
    for (k, loc) in mkeys(m)
        arr = floodfill(m, loc)
        res[k] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, loc)...)) for (k1, v1) in mkeys(m)])
    end
    
    arr = floodfill(m, hero(m))
    res['@'] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, hero(m))...)) for (k1, v1) in mkeys(m)])
    
    res
end

build_info (generic function with 1 method)

In [25]:
struct MazeState
    collected_keys::Set{Char}
    pos::Char
end

In [48]:
function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    nkeys = length(mkeys(m)) + 1
    info = build_info(m)
    memo = Dict{Tuple{Set{Char},Char}, Int}()
    s0 = (Set(Char[]), '@')
    memo[s0] = 0
    acc = [s0]
    while !isempty(acc)
#        sort!(acc, by = x -> -x[2]/length(x[1]))
        state = popfirst!(acc)
#        println(join(state[1]), ":", state[2], ":", memo[state])
        for (k, v) in info[state[2]]
            if !(k in state[1])
                if length(intersect(v[2], state[1])) == length(v[2])
                    new_keys = union(state[1], k, v[3])
                    new_state = (new_keys, k)
                    if !(new_state in keys(memo)) || memo[new_state] > memo[state] + v[1]
                        memo[new_state] = memo[state] + v[1]
                        push!(acc, new_state)
                    end
                end
            end
        end
    end
    
    return memo
end

part1 (generic function with 2 methods)

In [49]:
m = Maze(readlines("test2.txt"))
println.(readlines("test2.txt"))
res = part1("test2.txt")
# info = build_info(m)

#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################


Dict{Tuple{Set{Char},Char},Int64} with 10053 entries:
  (Set(['f', 'h', 'd', 'a', 'e', 'p', 'k', 'g', 'o', 'b']), 'o')           => 66
  (Set(['f', 'g', 'a', 'c', 'e', 'k', 'l', 'd', 'b']), 'l')                => 50
  (Set(['f', 'h', 'c', 'a', 'e', 'k', 'b']), 'h')                          => 41
  (Set(['f', 'h', 'c', 'l', 'd', 'p', 'e', 'm', 'b']), 'b')                => 69
  (Set(['f', 'j', 'a', 'c', 'e', 'k', 'h', 'm', 'd', 'b']), 'd')           => 69
  (Set(['f', 'd', 'e', 'h', 'i', 'c', 'p', 'm', 'g', 'l', 'b']), 'p')      => 78
  (Set(['f', 'e', 'h', 'j', 'k', 'a', 'c', 'p', 'm', 'g', 'b']), 'p')      => 80
  (Set(['f', 'h', 'a', 'g', 'e', 'p', 'l', 'd', 'o', 'b']), 'o')           => 68
  (Set(['n', 'f', 'g', 'a', 'i', 'c', 'h', 'b']), 'h')                     => 55
  (Set(['f', 'j', 'a', 'c', 'g', 'h', 'm', 'b']), 'm')                     => 48
  (Set(['f', 'h', 'd', 'a', 'e', 'j', 'o', 'b']), 'o')                     => 48
  (Set(['f', 'h', 'c', 'g', 'a', 'b']), 'a')           

In [51]:
m = Maze(readlines("test3.txt"))
println.(readlines("test3.txt"))
res = part1("test3.txt")

########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################


Dict{Tuple{Set{Char},Char},Int64} with 90 entries:
  (Set(['f', 'a']), 'a')                               => 17
  (Set(['f', 'd']), 'd')                               => 13
  (Set(['f', 'g', 'c', 'a', 'i', 'd', 'e']), 'e')      => 45
  (Set(['f', 'c', 'a', 'i']), 'i')                     => 29
  (Set(['d', 'c', 'e', 'a']), 'c')                     => 20
  (Set(['a', 'd', 'e']), 'd')                          => 31
  (Set(['f', 'c', 'a']), 'c')                          => 18
  (Set(['f', 'd', 'c', 'e', 'a']), 'f')                => 31
  (Set(['f', 'g', 'c', 'a', 'd', 'e', 'i', 'b']), 'b') => 63
  (Set(['f', 'g', 'a', 'd', 'e']), 'e')                => 39
  (Set(['f', 'd', 'c', 'a', 'e']), 'e')                => 33
  (Set(['f', 'a', 'e']), 'e')                          => 29
  (Set(['f', 'd', 'c', 'a']), 'c')                     => 20
  (Set(['f', 'd', 'a']), 'a')                          => 19
  (Set(['a', 'd']), 'd')                               => 29
  (Set(['f', 'g', 'a', 'd', 'c']),

In [54]:
m = Maze(readlines("input.txt"))
println.(readlines("input.txt"))
res = part1("input.txt")

#################################################################################
#...............#.......#...............#.........#.......#.......#.....#.......#
#.#########.#.###.#####.#.#.#######.#####.#######.#.#######.#.###.###.#.#.#####.#
#.#.#.....#.#.#...#...#..f#...#.....#...#...#.....#.........#...#.....#...#m....#
#.#.#.###.###.#.#####.#######.#######.#.#.#.#.#######.#########.###########.#####
#...#...#.#...#.......#.....#.#.......#.#.#.#...#...#...#.......#..h......#.....#
###.###.#.#.#########.#.###.#T#.#######.###.###.#B#.#.###.#######.#######.#####.#
#...#...#...#.....#...#...#.#...#.....#.#...#.#...#.#.#...#.........#...#.......#
#####.#######.###.#.###.#.#######.###.#.#.###.#####.###.#.###########.#.#######.#
#...#...#.....#.#...#.#.#.....#...#.....#.#.....#...#...#.#...........#.#.....#.#
#.#.###.#.#####.#####.#.#####.#.###.#####.#.###.#.###.#####.###########.#.###.#.#
#.#.....#.#.....#.........#...#...#.#...#.#...#..x#...W.....#.....#.....#.#...#.#
#.#######.###.#.

Dict{Tuple{Set{Char},Char},Int64} with 1222 entries:
  (Set(['s', 'a', 'k', 'p', 't', 'b', 'z']), 'z')                        => 948
  (Set(['w', 'd', 'e', 'o', 'h', 's', 'k', 't', 'a', 'c', 'p', 'm', 'x'… => 2340
  (Set(['s', 'z']), 's')                                                 => 104
  (Set(['x', 'c', 'a', 'k', 'p', 'e', 'b']), 'a')                        => 950
  (Set(['w', 'd', 'e', 'o', 'h', 's', 'k', 'c', 'p', 'm', 'x', 'v', 'u'… => 1852
  (Set(['w', 'e', 'o', 'h', 's', 'k', 't', 'a', 'c', 'p', 'm', 'x', 'v'… => 2924
  (Set(['x', 's', 'u', 'k', 'p', 'e', 'b']), 'e')                        => 708
  (Set(['x', 'k', 'p', 'e', 'b']), 'e')                                  => 580
  (Set(['w', 'e', 'o', 's', 'k', 't', 'a', 'c', 'p', 'm', 'z', 'x', 'v'… => 2762
  (Set(['u', 'l', 'f', 'w', 'a', 'c', 'e', 'p', 'o', 'z', 'g', 'j', 's'… => 3884
  (Set(['s', 'p', 'z']), 's')                                            => 164
  (Set(['j', 'f', 'a', 'c', 'e', 'p', 'o', 'z', 'x', 's', 'v',

In [56]:
minimum([v for (k, v) in res if length(k[1]) == 26])

4350

## Part 2

In [64]:
m = Maze(readlines("input.txt"))
println.(readlines("input.txt"))
h = hero(m)

#################################################################################
#...............#.......#...............#.........#.......#.......#.....#.......#
#.#########.#.###.#####.#.#.#######.#####.#######.#.#######.#.###.###.#.#.#####.#
#.#.#.....#.#.#...#...#..f#...#.....#...#...#.....#.........#...#.....#...#m....#
#.#.#.###.###.#.#####.#######.#######.#.#.#.#.#######.#########.###########.#####
#...#...#.#...#.......#.....#.#.......#.#.#.#...#...#...#.......#..h......#.....#
###.###.#.#.#########.#.###.#T#.#######.###.###.#B#.#.###.#######.#######.#####.#
#...#...#...#.....#...#...#.#...#.....#.#...#.#...#.#.#...#.........#...#.......#
#####.#######.###.#.###.#.#######.###.#.#.###.#####.###.#.###########.#.#######.#
#...#...#.....#.#...#.#.#.....#...#.....#.#.....#...#...#.#...........#.#.....#.#
#.#.###.#.#####.#####.#.#####.#.###.#####.#.###.#.###.#####.###########.#.###.#.#
#.#.....#.#.....#.........#...#...#.#...#.#...#..x#...W.....#.....#.....#.#...#.#
#.#######.###.#.

CartesianIndex(41, 41)

In [117]:
function prep_maze()
    m = Maze(readlines("input.txt"))
    h = hero(m)
    m.m[h[1], h[2]] = '#'
    m.m[h[1] + 1, h[2]] = '#'
    m.m[h[1] - 1, h[2]] = '#'
    m.m[h[1], h[2] + 1] = '#'
    m.m[h[1], h[2] - 1] = '#'
    m.m[h[1] + 1, h[2] + 1] = '1'
    m.m[h[1] - 1, h[2] + 1] = '2'
    m.m[h[1] + 1, h[2] - 1] = '3'
    m.m[h[1] - 1, h[2] - 1] = '4'
    
    m
end

function part2(m::Maze)
    res = Dict()
    for (k, loc) in mkeys(m)
        arr = floodfill(m, loc)
        res[k] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, loc)...)) for (k1, v1) in mkeys(m) if (arr[v1] != -1) && (arr[v1] != 0)])
    end
    
    for hh in '1':'4'
        arr = floodfill(m, find_element(m, hh))
        res[hh] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, find_element(m, hh))...)) for (k1, v1) in mkeys(m) if arr[v1] != -1])
    end
    
    info = res
    memo = Dict{Tuple{Set{Char}, Vector{Char}}, Int}()
    s0 = (Set(Char[]), collect('1':'4'))
    memo[s0] = 0
    acc = [s0]
    while !isempty(acc)
        state = popfirst!(acc)
#        println(join(state[1]), ":", state[2], ":", memo[state])
        for (i, loc) in enumerate(state[2])
            for (k, v) in info[loc]
                if !(k in state[1])
                    if length(intersect(v[2], state[1])) == length(v[2])
                        new_keys = union(state[1], k, v[3])
                        new_pos = copy(state[2])
                        new_pos[i] = k
                        new_state = (new_keys, new_pos)
                        if !(new_state in keys(memo)) || memo[new_state] > memo[state] + v[1]
                            memo[new_state] = memo[state] + v[1]
                            push!(acc, new_state)
                        end
                    end
                end
            end
        end
    end
    
    return memo
end

part2 (generic function with 1 method)

In [109]:
m = Maze(readlines("test4.txt"))
println.(readlines("test4.txt"))
part2(m)

###############
#d.ABC.#.....a#
######1#2######
###############
######3#4######
#b.....#.....c#
###############
a:CartesianIndex(14, 2)
c:CartesianIndex(14, 6)
d:CartesianIndex(2, 2)
b:CartesianIndex(2, 6)


Dict{Tuple{Set{Char},Array{Char,1}},Int64} with 9 entries:
  (Set(['a', 'c', 'b']), ['1', 'a', 'b', 'c'])      => 18
  (Set(['c', 'b']), ['1', '2', 'b', 'c'])           => 12
  (Set(Char[]), ['1', '2', '3', '4'])               => 0
  (Set(['c']), ['1', '2', '3', 'c'])                => 6
  (Set(['a', 'b']), ['1', 'a', 'b', '4'])           => 12
  (Set(['a', 'c']), ['1', 'a', '3', 'c'])           => 12
  (Set(['b']), ['1', '2', 'b', '4'])                => 6
  (Set(['a']), ['1', 'a', '3', '4'])                => 6
  (Set(['a', 'c', 'd', 'b']), ['d', 'a', 'b', 'c']) => 24

In [111]:
m = Maze(readlines("test5.txt"))
println.(readlines("test5.txt"))
res = part2(m)

#############
#DcBa.#.GhKl#
#.###1#2#I###
#e#d#####j#k#
###C#3#4###J#
#fEbA.#.FgHi#
#############
f:CartesianIndex(2, 6)
d:CartesianIndex(4, 4)
e:CartesianIndex(2, 4)
h:CartesianIndex(10, 2)
j:CartesianIndex(10, 4)
i:CartesianIndex(12, 6)
k:CartesianIndex(12, 4)
a:CartesianIndex(5, 2)
c:CartesianIndex(3, 2)
g:CartesianIndex(10, 6)
l:CartesianIndex(12, 2)
b:CartesianIndex(4, 6)


Dict{Tuple{Set{Char},Array{Char,1}},Int64} with 13 entries:
  (Set(['f', 'd', 'e', 'h', 'j', 'i', 'k', 'a', 'c', 'g', 'b']), ['e', 'j… => 28
  (Set(['a', 'c', 'd', 'b']), ['c', '2', 'd', '4'])                        => 9
  (Set(Char[]), ['1', '2', '3', '4'])                                      => 0
  (Set(['a', 'c', 'b']), ['c', '2', 'b', '4'])                             => 7
  (Set(['f', 'd', 'e', 'h', 'j', 'i', 'k', 'a', 'c', 'g', 'l', 'b']), ['e… => 32
  (Set(['a']), ['a', '2', '3', '4'])                                       => 2
  (Set(['f', 'a', 'c', 'd', 'e', 'b']), ['e', '2', 'f', '4'])              => 16
  (Set(['f', 'g', 'a', 'c', 'd', 'e', 'h', 'i', 'b']), ['e', 'h', 'f', 'i… => 24
  (Set(['f', 'g', 'a', 'c', 'd', 'e', 'b']), ['e', '2', 'f', 'g'])         => 19
  (Set(['a', 'c', 'd', 'e', 'b']), ['e', '2', 'd', '4'])                   => 12
  (Set(['a', 'b']), ['a', '2', 'b', '4'])                                  => 5
  (Set(['f', 'g', 'a', 'c', 'd', 'e', 'h', 'i', 'j', '

In [113]:
m = Maze(readlines("test6.txt"))
println.(readlines("test6.txt"))
res = part2(m)

#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba1#2BcIJ#
#############
#nK.L3#4G...#
#M###N#H###.#
#o#m..#i#jk.#
#############
n:CartesianIndex(2, 6)
f:CartesianIndex(4, 2)
d:CartesianIndex(2, 4)
e:CartesianIndex(6, 3)
o:CartesianIndex(2, 8)
h:CartesianIndex(10, 2)
j:CartesianIndex(10, 8)
i:CartesianIndex(8, 8)
k:CartesianIndex(11, 8)
a:CartesianIndex(5, 4)
c:CartesianIndex(10, 4)
m:CartesianIndex(4, 8)
g:CartesianIndex(2, 2)
l:CartesianIndex(12, 2)
b:CartesianIndex(4, 4)


Dict{Tuple{Set{Char},Array{Char,1}},Int64} with 70 entries:
  (Set(['h', 'a', 'c', 'e', 'b']), ['b', 'h', '3', '4'])                   => 12
  (Set(['f', 'd', 'e', 'h', 'j', 'i', 'k', 'a', 'c', 'g', 'b']), ['g', 'c… => 46
  (Set(['f', 'h', 'a', 'c', 'e', 'd', 'g', 'i', 'k', 'b']), ['g', 'c', '3… => 45
  (Set(['a', 'e', 'b']), ['e', '2', '3', '4'])                             => 5
  (Set(['f', 'h', 'a', 'c', 'e', 'd', 'g', 'b']), ['g', 'c', '3', '4'])    => 34
  (Set(['h', 'a', 'c', 'e', 'd', 'b']), ['d', 'h', '3', '4'])              => 14
  (Set(['a', 'c', 'e', 'd', 'b']), ['d', 'c', '3', '4'])                   => 8
  (Set(['a', 'c', 'e', 'b']), ['e', 'c', '3', '4'])                        => 7
  (Set(['n', 'f', 'd', 'e', 'h', 'j', 'i', 'k', 'a', 'c', 'g', 'l', 'b'])… => 60
  (Set(['a', 'e', 'b']), ['b', '2', '3', '4'])                             => 4
  (Set(['f', 'd', 'e', 'h', 'j', 'i', 'k', 'a', 'c', 'g', 'b']), ['g', 'c… => 52
  (Set(['h', 'a', 'i', 'e']), ['a', 'h', '3', 'i'])  

In [114]:
[v for (k, v) in res if length(k[1]) == 15]

2-element Array{Int64,1}:
 78
 72

In [119]:
m = prep_maze()
res = part2(m)

Dict{Tuple{Set{Char},Array{Char,1}},Int64} with 1655 entries:
  (Set(['n', 'f', 'w', 'o', 'h', 'r', 't', 'q', 'c', 'a'  …  'd', 'e', … => 3178
  (Set(['w', 'd', 'e', 'o', 'h', 's', 'k', 'c', 'a', 'p', 'm', 'z', 'x'… => 1774
  (Set(['f', 'w', 'e', 'o', 'j', 's', 'k', 't', 'c', 'a', 'p', 'z', 'x'… => 1740
  (Set(['f', 'w', 'o', 'i', 'r', 't', 'q', 'c', 'a', 'p'  …  'b', 'd', … => 2508
  (Set(['f', 'j', 'a', 'k', 'p', 't', 'b', 'z']), ['p', 't', 'k', 'j'])  => 850
  (Set(['f', 'e', 's', 't', 'k', 'q', 'a', 'p', 'z', 'x', 'v', 'u', 'l'… => 1938
  (Set(['w', 'd', 'e', 'o', 'h', 's', 'k', 't', 'c', 'a', 'p', 'm', 'z'… => 1850
  (Set(['x', 'c', 'v', 'k', 'p', 'e', 'u', 's', 'b', 'l']), ['l', 'v', … => 1054
  (Set(['x', 's', 'u', 'k', 'p', 'c', 'e', 'a', 'b', 'z']), ['p', 'b', … => 894
  (Set(['f', 'x', 'a', 'c', 'k', 'p', 't', 'e', 'b', 'z']), ['p', 'b', … => 1050
  (Set(['x', 's', 'a', 'k', 'p', 't', 'z', 'b']), ['p', 'b', 'x', 'a'])  => 754
  (Set(['q', 'b', 'f', 'w', 'j', 'c', 'a', 'e', 'p

In [121]:
minimum([v for (k, v) in res if length(k[1]) == 26])

2348

In [123]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1273


In [124]:
@benchmark part2($m)

BenchmarkTools.Trial: 
  memory estimate:  62.21 MiB
  allocs estimate:  733979
  --------------
  minimum time:     111.578 ms (5.80% GC)
  median time:      116.346 ms (6.31% GC)
  mean time:        116.967 ms (8.46% GC)
  maximum time:     123.315 ms (12.33% GC)
  --------------
  samples:          43
  evals/sample:     1

In [125]:
@benchmark part1()

BenchmarkTools.Trial: 
  memory estimate:  188.99 MiB
  allocs estimate:  1832612
  --------------
  minimum time:     205.483 ms (9.81% GC)
  median time:      209.794 ms (10.16% GC)
  mean time:        211.529 ms (10.88% GC)
  maximum time:     219.207 ms (13.30% GC)
  --------------
  samples:          24
  evals/sample:     1

In [101]:
Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, CartesianIndex(14, 2))...)) for (k1, v1) in mkeys(m) if (arr[v1] != -1) && (arr[v1] != 0)])

Dict{Char,Tuple{Any,Array{Char,1},Array{Char,1}}} with 0 entries

In [102]:
arr = floodfill(m, find_element(m, '1'))

15×7 Array{Int64,2}:
 -1  -1  -1  -1  -1  -1  -1
 -1   6  -1  -1  -1  -1  -1
 -1   5  -1  -1  -1  -1  -1
 -1   4  -1  -1  -1  -1  -1
 -1   3  -1  -1  -1  -1  -1
 -1   2  -1  -1  -1  -1  -1
 -1   1   0  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1

In [103]:
[(k1, v1) for (k1, v1) in mkeys(m) if arr[v1] != -1]

1-element Array{Tuple{Char,CartesianIndex{2}},1}:
 ('d', CartesianIndex(2, 2))

MethodError: MethodError: no method matching part2()
Closest candidates are:
  part2(!Matched::Maze) at In[108]:18

In [74]:
for i in 1:size(m.m)[1]
    for j in 1:size(m.m)[2]
        print(m.m[i, j])
    end
    print('\n')
end

#################################################################################
#.....#.#.........#.......#.............#.......#.................#...#.....#.C.#
#.###.#.#.###.###.#.###.###.#######L#.#######.#.###.###.#########E#.#.#.#.#.#.#.#
#.#.....#...#.#.#.#.#j..#.....#...#.#...#.....#...#.#...#...#...#...#v#.#.#...#.#
#.#########.#.#.#.###.###.#####.#.#.###.#.#######.#.#.###.#.#.#######.###.#####.#
#.#...#...#.#.#.#...#.....#.....#.#q#...#...#...#.#.#.#...#.#.#.....#...#.#...#.#
#.#.#.#.#.#.#.#.###.#######.#####.#.#.#####.#.###.#.#.#.###.#.#.#.#.###.#.###.#.#
#.#.#...#...#.#...#.........#...#.#.#...#...#.....#.#.....#.#...#.#...#.#.#...#o#
#.#.#########.#.#.#.#########.###.#####.#.###.###########.#.###.#.###.#.#.#.###.#
#.#.....#.....#.#...#.....#.#...#.......#.#.#.#.......#...#...#.#d#...#...#.....#
#.#####.#.#####.#####.###.#.#.#.#######.#.#.#.#.#####.#######.###.#.#######.#####
#...#...#.#.#.O.#.K...#.#.#...#.....#...#.#...#.#.N.#.......#.#...#.#.....#.....#
#.###.###.#.#.##

In [79]:
heros(m)
floodfill(m, heros(m)[4])

81×81 Array{Int64,2}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  …   -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  …   -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  …   -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1   -1   -1   -1   -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1      -1   -1  

In [34]:
typeof((Set(['a', 'b']), 'c'))

Tuple{Set{Char},Char}

In [21]:
a1 = Set([1, 2, 3])
a2 = union(a1, 4, [5, 6], 5, [1, 5, 6])

Set([4, 2, 3, 5, 6, 1])

In [19]:
pop!(a2)
pop!(a2)
(a1, a2)

(Set([2, 3, 1]), Set([3, 1]))

In [29]:
res = build_info(m)
keys(res['@'])

Base.KeySet for a Dict{Char,Tuple{Int64,Array{Char,1}}} with 6 entries. Keys:
  'f'
  'a'
  'c'
  'd'
  'e'
  'b'

In [28]:
intersect(res['@']['f'][2], ['a', 'c', 'x'])

2-element Array{Char,1}:
 'c'
 'a'

In [59]:
[(Set([1]), 1), (Set([2]), 1)] == [(Set([1]), 1), (Set([2]), 1)]

true

In [60]:
typeof([(Set([1]), 1), (Set([2]), 1)])

Array{Tuple{Set{Int64},Int64},1}

In [ ]:
struct Maze
    m::Array{Char, 2}
    score::Int
end

Maze(s::Vector{T}, score::Int) where T <: String = Maze(hcat(collect.(s)...), score)
Maze(s::Vector{T}) where T <: String = Maze(s, 0)

h(m::Maze, l1, l2) = sum(abs.(Tuple(l1 - l2)))

val(m::Maze, l) = m.m[l]

allowed(m::Maze) = [collect('a':'z'); '@'; '.']

upper(c::Char) = c + 'A' - 'a'

function neighbours(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[val(m, l + d) in allowed(m) for d in dirs]]]
end

function neighbours2(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[val(m, l + d) != '#' for d in dirs]]]
end

function reconstruct_path(came_from::Dict{T, T}, current::T) where T
    path = T[]
    prev = current
    while current in keys(came_from)
        push!(path, current)
        current = came_from[current]
        prev = current
    end
    
    reverse(path)
end

function least(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node][2] < m
            m = scores[node][2]
            res = node
        end
    end
    
    res
end

function astar(m::U, start::T, goal::T) where {T, U}
    openset = Set([start])
    scores = Dict(start => [0, h(m, start, goal)])
    came_from = Dict{T, T}()
    current = start
    while !isempty(openset)
        current = least(openset, scores)
        if current == goal
            return reconstruct_path(came_from, goal)
        end
        
        delete!(openset, current)
        for neighbour in neighbours(m, current)
            tentative_g_score = scores[current][1] + 1
            neighbour_g_score = get!(scores, neighbour, [-1, h(m, neighbour, goal)])[1]
            if neighbour_g_score < 0 || neighbour_g_score > tentative_g_score
                came_from[neighbour] = current
                scores[neighbour][1] = tentative_g_score
                if !(neighbour in openset) push!(openset, neighbour) end
            end
        end
    end
                
    if current != goal return T[] end
end

heur(hdict, c, ks) = sum([v for (k, v) in hdict[c] if k in ks])

In [ ]:
function floodfill(m::Maze)
    arr = fill(-1, size(m.m)...)
    start = hero(m)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function floodfill2(m::Maze, start)
    arr = fill(-1, size(m.m)...)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours2(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function build_proto(m::Maze)
    res = Dict()
    for (k, loc) in mkeys(m)
        arr = floodfill2(m, loc)
        res[k] = Dict([(k1, arr[v1]) for (k1, v1) in mkeys(m)])
    end
    
    arr = floodfill2(m, hero(m))
    res['@'] = Dict([(k1, arr[v1]) for (k1, v1) in mkeys(m)])
    
    res
end

function least(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node][2] < m
            m = scores[node][2]
            res = node
        end
    end
    
    res
end

function least2(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node[2]] < m
            m = scores[node[2]]
            res = node
        end
    end
end

function xstar(m, start = "@")
    hdict = build_proto(m)
    openset = Set([(m, start)])
    scores = Dict(start => heur(hdict, start[end], keys(mkeys(m))))
    came_from = Dict()
    current = start
    while !isempty(openset)
        current = least2(openset, scores)
        current_maze, current_path, current_key = current
        
        if isempty(mkeys(current_maze))
             return (came_from, current_path, current_key)
        end
        
        delete!(openset, current)
        
        paths = floodfill(current_maze)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(current_maze)])
        
        for (k, l) in available_keys
            new_m = copy(current_maze.m)
            new_m[hero(current_maze)] = '.'
            new_m[mkeys(current_maze)[k]] = '@'
            if uppercase(k) in keys(doors(current_maze)) new_m[doors(current_maze)[uppercase(k)]] = '.' end
            push!(openset, (Maze(new_m, state.score + l), current_path * k))
        end
        
        for (k, l) in available_keys
            tentative_g_score = scores[(current_path, current_key)][1] + l
            neighbour_g_score = get!(scores, (current_path*string(k), ))
        end
        
        for neighbour in neighbours(m, current)
            tentative_g_score = scores[current][1] + 1
            neighbour_g_score = get!(scores, neighbour, [-1, h(m, neighbour, goal)])[1]
            if neighbour_g_score < 0 || neighbour_g_score > tentative_g_score
                came_from[neighbour] = current
                scores[neighbour][1] = tentative_g_score
                if !(neighbour in openset) push!(openset, neighbour) end
            end
        end
    end
                
    if current != goal return T[] end
end

In [ ]:
find_element(m::Maze, el::Char) = get(CartesianIndices(m.m)[m.m .== el], 1, nothing)
find_elements(m::Maze, els) = [(x, find_element(m, x)) for x in els if find_element(m, x) != nothing]
mkeys(m::Maze) = Dict(find_elements(m, 'a':'z'))
doors(m::Maze) = Dict(find_elements(m, 'A':'Z'))
hero(m::Maze) = find_element(m, '@')

In [ ]:
m = Maze(readlines("input.txt"))
sum(values(build_proto(m)['u']))

In [ ]:
arr = floodfill(m)

In [ ]:
filter(x -> x[2] > 0, [(k, arr[v]) for (k, v) in mkeys(m)])
# [(k, arr[v]) for (k, v) in mkeys(m)]

In [ ]:
## A-star???

function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    v1 = []
    acc = [(m, "")]
    min_score = -1
    while !isempty(acc)
        state, path = pop!(acc)
        println(join(path), " : ", state.score, " : ", min_score, " : ", length(v1))
        paths = floodfill(state)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(state)])
        if isempty(available_keys)
            if isempty(mkeys(state))
                if min_score < 0 || state.score < min_score 
                    min_score = state.score 
                    push!(v1, (path, state.score))
                    continue
                end
            end
        end
        for (k, l) in available_keys
            if min_score > 0 && (state.score + l) > min_score continue end
            new_m = copy(state.m)
            new_m[hero(state)] = '.'
            new_m[mkeys(state)[k]] = '@'
            if uppercase(k) in keys(doors(state)) new_m[doors(state)[uppercase(k)]] = '.' end
            push!(acc, (Maze(new_m, state.score + l), path * k))
        end
    end
    
    v1
end

In [ ]:
h1(m::Maze, loc) = sum([sum(abs.(Tuple(loc - v))) for (k, v) in mkeys(m)])

In [ ]:
function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    v1 = []
    acc = [(m, "")]
    min_score = -1
    while !isempty(acc)
        state, path = pop!(acc)
        println(join(path), " : ", state.score, " : ", min_score, " : ", length(v1))
        paths = floodfill(state)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(state)])
        if isempty(available_keys)
            if isempty(mkeys(state))
                if min_score < 0 || state.score < min_score 
                    min_score = state.score 
                    push!(v1, (path, state.score))
                    continue
                end
            end
        end
        for (k, l) in available_keys
            if min_score > 0 && (state.score + l) > min_score continue end
            new_m = copy(state.m)
            new_m[hero(state)] = '.'
            new_m[mkeys(state)[k]] = '@'
            if uppercase(k) in keys(doors(state)) new_m[doors(state)[uppercase(k)]] = '.' end
            push!(acc, (Maze(new_m, state.score + l), path * k))
        end
    end
    
    v1
end

In [ ]:
res = part1("test2.txt")

In [ ]:
m = Maze(readlines("input.txt"))
hdict = build_proto(m)
heur(hdict, 'a', keys(mkeys(m)))
# 5806
# sum(values(build_proto(m)['@']))

In [ ]:
h1(m, hero(m))

In [ ]:
function f1(m)
#     mkeys(m)
#     hero(m)
    astar(m, hero(m), mkeys(m)['a'])
    # available_keys = filter( x -> x[2] > 0, [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(m)])
end

In [ ]:
using BenchmarkTools

@benchmark f1($m)

In [ ]:
mkeys(m)

In [ ]:
Profile.print()

In [ ]:
Profile.print(format=:flat)

In [ ]:
x0 = hero(m)
x1 = neighbours(m, hero(m))[4]
astar(m, x0, x1)

In [ ]:
ad = [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(m)]
ad = filter( x -> x[2] > 0, ad)

In [ ]:
find_element(m, 'a')

In [ ]:
m.m[find_element(m, 'a')] = '.'

In [ ]:
@benchmark mkeys($m)

In [ ]:
b = copy(a)

In [ ]:
b[2, 2] = 5

In [ ]:
a

In [ ]:
b

In [ ]:
dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
#    l .+ dirs[val(m, l + d), allowed(m)) for d in dirs]

In [ ]:
dirs[[val(m, hero(m) + d) in allowed(m) for d in dirs]]

In [ ]:
2 in [3, 4, 2]

In [ ]:
paths = floodfill(m)
available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(m)])

In [ ]:
using BenchmarkTools

@benchmark neighbours($m, $hero(m))

In [ ]:
upper(c) = ('A' - 'a') + c

In [ ]:
upper('c')

In [ ]:
fill(-1, size(m.m)...)

In [ ]:
function ff(m)
    me = hero(m)
    available_keys = filter( x -> x[2] > 0, [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(state)])
end

In [ ]:
heur(hdict, c, ks) = sum([v for (k, v) in hdict[c] if k in ks])

In [ ]:
"abc"[end]